In [1]:
import os
import numpy as np
from dotenv import load_dotenv

import faiss
import google.generativeai as genai

C:\Users\PC\Code\Python\Publish-To-Git\RAG-With-LangChain-And-FAISS\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## A.　載入 .env 取得 LLM API Key

In [2]:
load_dotenv()
# print(os.getenv('GEMINI_API_KEY'))
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

# 

## B.　RAG 實作

### I.　建立向量資料庫

In [3]:
## 使用 Google Gemini 生成文字的向量嵌入
def get_gemini_embedding(text):
    response = genai.embed_content(model="models/embedding-001", content=text, task_type="retrieval_document")
    return response["embedding"]

In [4]:
documents = [
    "Google Gemini 是一款大型語言模型（LLM），由 Google DeepMind 開發，適用於各種自然語言處理（NLP）任務。",
    # "RAG（檢索增強生成）結合了檢索與 AI 生成，提升模型回答的準確度。",
    "RAG是知名品牌製作的高效清潔抹布，通常市售30TWD，基本每戶人家都會有一條RAG放置廚房做使用",
    "FAISS 是一款高效的向量搜尋庫，適合用於 AI 相關應用。",
]

In [5]:
embeddings = np.array([get_gemini_embedding(doc) for doc in documents], dtype=np.float32)
print("Embedding Shape：", embeddings.shape)
print("Content Top 5：", embeddings[:5])

Embedding Shape： (3, 768)
Content Top 5： [[ 0.04012055 -0.01553916 -0.05447042 ...  0.02586641 -0.01795555
   0.01585375]
 [ 0.06144911 -0.00659113 -0.05377533 ...  0.00689761  0.00225536
   0.00023795]
 [ 0.0564121  -0.01196416 -0.06638404 ...  0.01994789 -0.00251821
   0.0266339 ]]


In [6]:
### 建立 FAISS 索引
dimension = embeddings.shape[1]  # 嵌入向量的維度
index = faiss.IndexFlatL2(dimension) # 使用 L2 距離索引
index.add(embeddings) # 新增向量資料

### 儲存 FAISS 索引
faiss.write_index(index, "faiss_index.bin")

print("✅ FAISS 向量資料庫已建立！")

✅ FAISS 向量資料庫已建立！


### II.　使用 RAG 進行增強式答覆

In [7]:
### 從 FAISS 檢索最相關的資料
def retrieve_from_faiss(query, top_k=2):
    index = faiss.read_index("faiss_index.bin") # 載入索引
    query_vector = np.array([get_gemini_embedding(query)], dtype=np.float32) # 轉換為向量
    distances, indices = index.search(query_vector, top_k) # 搜索最相關的 K 個結果
    return [documents[i] for i in indices[0]] # 返回對應的文件

In [8]:
### RAG 流程：檢索 + 生成回答
model = "gemini-1.5-flash"

def rag_query(query):
    relevant_docs = retrieve_from_faiss(query)
    context = "\n".join(relevant_docs)

    prompt = f"""
    我希望你扮演 "基於事實回答的顧問" 的角色。
    - 請根據以下資訊回答問題 : {context}
    - 問題 : {query}
    - 遵守下列規則 : 
      * 請勿幻覺答覆
      * 繁體中文回覆
    """
    
    client = genai.GenerativeModel(model)
    res = client.generate_content(prompt)
    return res.text

def norm_query(query):
    prompt = f"""
    我希望你扮演 "基於事實回答的顧問" 的角色。
    - 問題 : {query}
    - 遵守下列規則 : 
      * 請勿幻覺答覆
      * 繁體中文回覆
    """
    
    client = genai.GenerativeModel(model)
    res = client.generate_content(prompt)
    return res.text

# 

## C.　成果展示

### I.　什麼是 RAG？

In [9]:
norm_query("什麼是 RAG？")

'RAG 指的是 Retrieval Augmented Generation，中文翻譯為**檢索增強生成**。  它是一種人工智慧技術，結合了資訊檢索和大型語言模型 (LLM) 的能力，來改善 LLM 的輸出結果。\n\n具體來說，RAG 系統會先根據使用者提出的問題，從一個已建立的知識庫 (例如，資料庫、文件集合或網頁) 中檢索相關資訊。然後，它會將這些檢索到的資訊，與使用者的問題一起提供給 LLM。LLM  並非僅僅依賴其內部訓練的知識，而是利用檢索到的資訊來生成更準確、更相關、更具事實依據的回答。\n\n因此，RAG 的核心優勢在於它可以克服 LLM  知識庫有限的缺點，並提升其回答的準確性和可信度。  它尤其適用於需要處理大量特定領域知識或最新資訊的應用場景。\n'

In [10]:
rag_query("什麼是 RAG？")

'根據提供的資訊，RAG指的是一個知名品牌生產的高效清潔抹布，售價約為30元新台幣，普遍存在於台灣家庭廚房中。  它與向量搜尋庫FAISS無關。\n'

### II.　RAG 是抹布嗎? 超商可以買到它嗎?

In [11]:
norm_query("RAG 是抹布嗎? 超商可以買到它嗎?")

'RAG 不是抹布。RAG 指的是 Retrieval Augmented Generation，是一種利用外部知識庫來增強大型語言模型生成能力的技術。它是一種人工智慧的概念，而非實體物品。因此，超商無法買到它。\n'

In [12]:
rag_query("RAG 是抹布嗎? 超商可以買到它嗎?")

'根據提供的資訊，RAG是一種市售價格約30元台幣的清潔抹布，常見於一般住家廚房使用。  然而，資訊中並沒有說明它是否在超商販售。因此，我無法確認超商是否可以買到RAG。\n'

### III.　什麼是 Agent？

In [13]:
norm_query("什麼是 Agent？")

'在電腦科學和人工智慧領域，「Agent」指的是一個能夠自主地感知其環境並採取行動以達成目標的實體。  這是一個廣泛的概念，涵蓋範圍很廣，從簡單的程式到複雜的機器人系統。  它沒有單一的定義，取決於上下文而有所不同。\n\n更具體來說，一個Agent通常具備以下幾個特點：\n\n* **自主性 (Autonomy):**  Agent可以獨立運作，不需要持續的人工干預。  它可以根據自身的感知和內部狀態做出決策。\n\n* **目標導向 (Goal-oriented):** Agent有明確或隱含的目標，其行動都是為了達成這些目標。\n\n* **感知能力 (Perception):** Agent可以感知其環境，例如接收來自感測器或網路的輸入。\n\n* **行動能力 (Action):** Agent可以採取行動來影響其環境，例如移動、操作物件或發送訊息。\n\n* **理性 (Rationality):** Agent會盡可能地採取最能達成其目標的行動，儘管這並不一定意味著總是做出最佳選擇，因為資訊可能不完整或計算能力有限。\n\n\nAgent的例子包括：\n\n* **軟體Agent:** 例如搜尋引擎的爬蟲、網路聊天機器人、電子郵件過濾器等。\n* **機器人Agent:** 例如自動駕駛汽車、工業機器人、家用機器人等。\n* **人類Agent:**  在某些模型中，人類也可以被視為Agent，例如在多Agent系統中。\n\n\n重要的是要記住，「Agent」是一個抽象的概念，其具體實現方式會因應不同的應用場景而有所不同。  沒有單一的、放諸四海皆準的Agent定義。\n'

In [14]:
rag_query("什麼是 Agent？")

'根據提供的資訊，我無法回答「什麼是 Agent？」。  提供的文字只說明了FAISS 是一個向量搜尋庫，以及 RAG 是一種清潔抹布。  這兩個概念與「Agent」沒有任何直接關聯。  要回答「什麼是 Agent？」，需要更多關於「Agent」的背景資訊或定義。\n'